In [1]:
import pandas as pd
import numpy as np

In [41]:
salary_data = pd.read_excel("salaire.xls", skiprows=5)
immigration_data = pd.read_excel("immigration.xls", skiprows=10)
socio_data = pd.read_excel("socioprofessional.xls", skiprows=10)

### First, as usual, we convert unicode to string.

In [3]:
import unicodedata

def unicode_to_string(word):
    if pd.isnull(word):
        return 
    else:
        return unicodedata.normalize('NFKD', word).encode('ascii','ignore')
    
def data_to_string(data):
    new_data = data.copy()
    
    col_title_unicode = new_data.columns.values.tolist()
    col_title = map(unicode_to_string,col_title_unicode)
    new_data.columns = col_title
    
    for col in new_data.columns:
        not_nan_index = [not ind for ind in new_data[col].isnull()]
        not_nan_value = new_data[col][not_nan_index]
        if type(not_nan_value.iloc[0]) == unicode: #check the first not-NaN value
            new_data[col] = map(unicode_to_string,new_data[col])
            
    return new_data

In [43]:
salary_data = data_to_string(salary_data)
immigration_data = data_to_string(immigration_data)
socio_data = data_to_string(socio_data)

### Now we will rename the column headers so that we have at least 1 common column for all the data frames. 

In [5]:
def rename_column(data): 
    new_data = data.copy()
    col_title = new_data.columns.tolist()
    col_title[0] = 'Code Insee' # this will be the pivot column for merging
    col_title[1] = 'Libelle de la commune'
    new_data.columns = col_title
    return new_data

In [44]:
salary_data = rename_column(salary_data)
immigration_data = rename_column(immigration_data)
socio_data = rename_column(socio_data)

### We still have problem with departemental code of Corse because they are not in numerical form (2A... or 2B...).

In [7]:
def to_digit(string):
    new_string = list(string)[:]
    for x in xrange(len(new_string)): 
        if not new_string[x].isdigit():
            new_string[x] = '0'
    return int(''.join(new_string))

In [8]:
def replace_insee_code(data): #replace 2AXXX or 2BXXX by 20XXX 
        new_data = data.copy()
        insee_list = new_data['Code Insee'].tolist() 
        new_insee_list = [int(code) if code.isdigit() else to_digit(code) for code in insee_list]
        new_data['Code Insee'] = new_insee_list
        return new_data

In [45]:
salary_data = replace_insee_code(salary_data)
immigration_data = replace_insee_code(immigration_data)
socio_data = replace_insee_code(socio_data)

### Now we try to merge the 3 data frames: right_party_vote, immigration_data, salary_data

In [10]:
right_party_vote = pd.read_excel("right_party_vote.xlsx")

In [11]:
immigration_salary = pd.merge(immigration_data, salary_data, on = ['Code Insee','Libelle de la commune'], how = 'outer')

In [54]:
immigration_salary_socio = pd.merge(socio_data, immigration_salary, on = ['Code Insee','Libelle de la commune'], how = 'outer')

In [55]:
full_data = pd.merge(right_party_vote, immigration_salary_socio, on = 'Code Insee', how = 'outer')